In [11]:
import pandas as pd
import numpy as np
import io
!pip install chardet
import chardet

In [13]:
# Thiết lập
file_path = r"C:\Users\Dell\Downloads\TradeData_10_28_2025_21_41_43.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

with open(file_path, "rb") as f:
    rawdata = f.read(200000)
    result = chardet.detect(rawdata)
print("Mã hóa dự đoán:", result)

Mã hóa dự đoán: {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [14]:
df = pd.read_csv(file_path, encoding="ISO-8859-1", low_memory=False)
print(df.head())


  typeCode  freqCode  refPeriodId  refYear  refMonth  period reporterCode reporterISO reporterDesc flowCode  flowDesc partnerCode partnerISO  partnerDesc partner2Code partner2ISO partner2Desc classificationCode  classificationSearchCode  isOriginalClassification                         cmdCode  cmdDesc  aggrLevel isLeaf customsCode  customsDesc  mosCode    motCode  motDesc qtyUnitCode  qtyUnitAbbr    qty  isQtyEstimated altQtyUnitCode  altQtyUnitAbbr  altQty  isAltQtyEstimated  netWgt  isNetWgtEstimated  grossWgt  isGrossWgtEstimated  cifvalue      fobvalue  primaryValue  legacyEstimationFlag  isReported  isAggregate
C        A  20200101         2020       52      2020     156          CHN       China            M   Import         0         W00      World            0          W00       World           H5                 HS                      True                      8542  Electronic integrated circuits        4      False    C00   TOTAL CPC            0        0  TOTAL MOT       -1

- Shape & Info

In [18]:
if 'df' in locals():
    print(f"--- 1. Kích thước (Shape) ---")
    print(f"Số hàng: {df.shape[0]}, Số cột: {df.shape[1]}")
    
    print(f"--- 2. Thông tin cột (Info) ---")
    import io
    buf = io.StringIO()
    df.info(buf=buf)
    print(buf.getvalue())
else:
    print("LỖI: Biến 'df' chưa được tải")

--- 1. Kích thước (Shape) ---
Số hàng: 13689, Số cột: 47
--- 2. Thông tin cột (Info) ---
<class 'pandas.core.frame.DataFrame'>
Index: 13689 entries, C to C
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  13689 non-null  object 
 1   freqCode                  13689 non-null  int64  
 2   refPeriodId               13689 non-null  int64  
 3   refYear                   13689 non-null  int64  
 4   refMonth                  13689 non-null  int64  
 5   period                    13689 non-null  int64  
 6   reporterCode              13689 non-null  object 
 7   reporterISO               13689 non-null  object 
 8   reporterDesc              13689 non-null  object 
 9   flowCode                  13689 non-null  object 
 10  flowDesc                  13689 non-null  int64  
 11  partnerCode               13689 non-null  object 
 12  partnerISO                13689 non-

- Thống kê mô tả

In [ ]:
if 'df' in locals():
    print(f"--- 3. Thống kê mô tả (Describe) cho các cột số ---")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    if not numeric_cols.empty:
        print(df[numeric_cols].describe().T)
    else:
        print("Không tìm thấy cột số nào.")
else:
    print("LỖI: Biến 'df' chưa được tải")

--- 3. Thống kê mô tả (Describe) cho các cột SỐ ---
                            count          mean           std         min         25%           50%           75%           max
freqCode                  13689.0  2.021954e+07  1.377720e+04  20200101.0  20210101.0  2.022010e+07  2.023010e+07  2.024010e+07
refPeriodId               13689.0  2.021944e+03  1.377720e+00      2020.0      2021.0  2.022000e+03  2.023000e+03  2.024000e+03
refYear                   13689.0  5.200000e+01  0.000000e+00        52.0        52.0  5.200000e+01  5.200000e+01  5.200000e+01
refMonth                  13689.0  2.021944e+03  1.377720e+00      2020.0      2021.0  2.022000e+03  2.023000e+03  2.024000e+03
period                    13689.0  4.837865e+02  2.740142e+02       156.0       156.0  4.100000e+02  8.420000e+02  8.420000e+02
flowDesc                  13689.0  4.392403e+02  2.509448e+02         0.0       218.0  4.420000e+02  6.620000e+02  8.990000e+02
partnerDesc               13689.0  0.000000e+00  0.0

In [22]:
if 'df' in locals():
    print(f"--- 4. Thống kê mô tả (Describe) cho các cột chữ (Object/Bool) ---")
    object_cols = df.select_dtypes(include=['object', 'bool']).columns
    
    if not object_cols.empty:
        print(df[object_cols].describe().T)
    else:
        print("Không tìm thấy cột chữ (object) hoặc boolean nào.")
else:
    print("LỖI: Biến 'df' chưa được tải")

--- 4. Thống kê mô tả (Describe) cho các cột chữ (Object/Bool) ---
                          count unique                             top   freq
typeCode                  13689      1                               A  13689
reporterCode              13689      5                             CHN   4053
reporterISO               13689      5                           China   4053
reporterDesc              13689      2                               X   7413
flowCode                  13689      2                          Export   7413
partnerCode               13689    231                             W00    144
partnerISO                13689    232                           World    144
partner2Code              13689      1                             W00  13689
partner2ISO               13689      1                           World  13689
partner2Desc              13689      2                              H6   8091
classificationCode        13689      1                              HS  136

- Kiểm tra các cột quan trọng

In [27]:
if 'df' in locals():
    print(f"--- 5. Kiểm tra các cột quan trọng (SỬA LỖI LẦN 2) ---")
    
    # === 1. Lấy cột Năm ===
    if 'refMonth' in df.columns:
        print(f"Phạm vi năm: {df['refMonth'].unique()}")
    else:
        print("LỖI: Không tìm thấy cột 'refMonth'.")

    
    print(f"\nCác hướng giao dịch (flowCode) [M=Import, X=Export]:")
    if 'flowCode' in df.columns:
        print(df['flowCode'].value_counts())
    
    
    # === 2. Lấy cột Mã sản phẩm ===
    print(f"\nCác mã sản phẩm (cmdCode) - Top 5:")
    if 'cmdCode' in df.columns:
        cmd_counts = df['cmdCode'].value_counts()
        print(cmd_counts.head())
        
        if '85' in cmd_counts.index or 85 in cmd_counts.index:
            count_85 = cmd_counts.loc[cmd_counts.index.isin(['85', 85])].sum()
            print(f"==> TÌM THẤY: {count_85} / {df.shape[0]} dòng là mã '85' (Điện tử)")
        else:
             print("==> LƯU Ý: Không tìm thấy mã '85' (Điện tử) trong dữ liệu.")
             
            
    # === 3. Lấy đúng cột Quốc gia (dùng cột ISO) ===
    print(f"\nQuốc gia Báo cáo (reporterISO) - Top 5:")
    if 'reporterISO' in df.columns:
        print(df['reporterISO'].value_counts().head())

    print(f"\nQuốc gia Đối tác (partnerISO) - Top 5:")
    if 'partnerISO' in df.columns:
        print(df['partnerISO'].value_counts().head())
else:
    print("LỖI: Biến 'df' chưa được tải.")

--- 5. Kiểm tra các cột quan trọng (SỬA LỖI LẦN 2) ---
Phạm vi năm: [2020 2021 2022 2023 2024]

Các hướng giao dịch (flowCode) [M=Import, X=Export]:
flowCode
Export    7413
Import    6276
Name: count, dtype: int64

Các mã sản phẩm (cmdCode) - Top 5:
cmdCode
Electronic integrated circuits                                                                                                                                                             5245
Electronic integrated circuits; n.e.c. in heading no. 8542                                                                                                                                 4353
Electronic integrated circuits; processors and controllers, whether or not combined with memories, converters, logic circuits, amplifiers, clock and timing circuits, or other circuits    4091
Name: count, dtype: int64
==> LƯU Ý: Không tìm thấy mã '85' (Điện tử) trong dữ liệu.

Quốc gia Báo cáo (reporterISO) - Top 5:
reporterISO
China            4053
USA   

- Kiểm tra dữ liệu thuế

In [29]:
if 'df' in locals():
    print(f"--- 6. KIỂM TRA DỮ LIỆU THUẾ QUAN (TARIFF) ---")
    
    # Tạo một danh sách các cột có tên chứa các từ khóa về thuế
    possible_tariff_cols = [col for col in df.columns 
                            if 'tariff' in col.lower() 
                            or 'duty' in col.lower() 
                            or 'tax' in col.lower() 
                            or 'rate' in col.lower()]

    if len(possible_tariff_cols) > 0:
        print(f"==> PHÁT HIỆN: Tìm thấy các cột có thể liên quan đến thuế: {possible_tariff_cols}")
        print("Vui lòng kiểm tra các cột này kỹ hơn ở bước sau.")
    else:
        print("==> LƯU Ý QUAN TRỌNG: Không tìm thấy cột nào chứa dữ liệu THUẾ SUẤT (Tariff/Duty).")
        print("Các cột như 'cifvalue', 'fobvalue' là TỔNG GIÁ TRỊ GIAO DỊCH ($), không phải thuế suất (%).")

    print("\n--- Kiểm tra hoàn tất ---")
else:
    print("LỖI: Biến 'df' chưa được tải.")

--- 6. KIỂM TRA DỮ LIỆU THUẾ QUAN (TARIFF) ---
==> LƯU Ý QUAN TRỌNG: Không tìm thấy cột nào chứa dữ liệu THUẾ SUẤT (Tariff/Duty).
Các cột như 'cifvalue', 'fobvalue' là TỔNG GIÁ TRỊ GIAO DỊCH ($), không phải thuế suất (%).

--- Kiểm tra hoàn tất ---
